In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

from copy import deepcopy
import itertools
import pickle

import pandas as pd
from scipy.io import savemat, loadmat
import seaborn as sns
from sklearn.metrics import (log_loss, mean_squared_error, roc_curve, auc,
                             precision_recall_fscore_support, confusion_matrix)
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_validate
from tqdm import tqdm, trange, tqdm_notebook as tqdmn

from BEKML import BEMKL, plot_distplot
from utils import poly_kernel, gauss_kernel, scoring, plot_kernel_importances, plot_compare_models

sns.set(style='ticks', context='talk')
np.set_printoptions(precision=4, linewidth=100)

Populating the interactive namespace from numpy and matplotlib


/home/mtambos/anaconda/envs/pml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('data/wdbc.csv', names=list(range(32)), index_col=0)
display(data.head())
data.describe()

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,,,,,,,,,,,,,,,,,,,,,
842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


,2,3,4,5,6,7,8,9,10,11,...,22,23,24,25,26,27,28,29,30,31
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [3]:
X = data.iloc[:, 1:].copy()
y = data.iloc[:, 0].copy()
N, D = X.shape
y.loc[y == 'B'] = -1
y.loc[y == 'M'] = 1
y = y.astype(int)
display(X.shape, y.shape, y.mean())

(569, 30)

(569,)

-0.2548330404217926

In [4]:
proc_data = data.copy()
proc_data.iloc[:, 0] = y
proc_data = proc_data.loc[:, list(proc_data.columns[1:]) + [proc_data.columns[0]]]
proc_data.to_csv('data/proc_wdbc.csv')
proc_data.head()

,2,3,4,5,6,7,8,9,10,11,...,23,24,25,26,27,28,29,30,31,1
0,,,,,,,,,,,,,,,,,,,,,
842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [4]:
rbf_init, rbf_end = -3, 7
ply_init, ply_end = 1, 4

kernel_attrs = [('rbf', 'all', i) for i in range(rbf_init, rbf_end)]
kernels = [lambda A, B: gauss_kernel(A, B, 2**i)
           for i in range(rbf_init, rbf_end)]

kernel_attrs += [('poly', 'all', i) for i in range(ply_init, ply_end)]
kernels += [lambda A, B: poly_kernel(A, B, 1, i)
            for i in range(ply_init, ply_end)]

kernel_attrs += [('rbf', j, i) for i in range(rbf_init, rbf_end)
                 for j in range(D)]
kernels += [lambda A, B: gauss_kernel(A[:, j:j+1], B[:, j:j+1], 2**i)
            for i in range(rbf_init, rbf_end) for j in range(D)]

kernel_attrs += [('poly', j, i) for i in range(ply_init, ply_end)
                 for j in range(D)]
kernels += [lambda A, B: poly_kernel(A[:, j:j+1], B[:, j:j+1], 1, i)
            for i in range(ply_init, ply_end) for j in range(D)]
len(kernels), len(kernel_attrs)

(403, 403)

In [5]:
X_train, X_test, y_train, y_test =\
        train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

In [6]:
normalizer = Normalizer()
X_train_norm = normalizer.fit_transform(X_train)
X_test_norm = normalizer.fit_transform(X_test)

# Baseline Model

In [14]:
max_iter = 200
base_model = BEMKL(kernels=kernels, hyp_lambda_alpha=1, hyp_lambda_beta=1,
                   hyp_gamma_alpha=1, hyp_gamma_beta=1,
                   hyp_omega_alpha=1, hyp_omega_beta=1,
                   e_null_thrsh=1e-2, a_null_thrsh=1e-2,
                   filter_kernels=False, filter_sv=False, verbose=False,
                   max_iter=max_iter, hyperopt_enabled=False, calculate_bounds=False)
base_model = make_pipeline(Normalizer(), base_model)

scoring.iteration = 0
scoring.stats = []
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=5)
base_cv_results = cross_validate(base_model, X, y, cv=folds, scoring=scoring)
base_stats = deepcopy(scoring.stats)
base_cv_results, base_stats

0 - Kernels: 13/403 (0.03225806451612903). SV: 374/426 (0.8779342723004695). Mean e: 0.0146. Median e: 0.0077. Std e: 0.0752. 
1 - Kernels: 13/403 (0.03225806451612903). SV: 327/427 (0.765807962529274). Mean e: 0.0151. Median e: 0.0082. Std e: 0.0755. 
2 - Kernels: 13/403 (0.03225806451612903). SV: 352/427 (0.8243559718969555). Mean e: 0.0150. Median e: 0.0076. Std e: 0.0809. 
3 - Kernels: 13/403 (0.03225806451612903). SV: 389/427 (0.9110070257611241). Mean e: 0.0163. Median e: 0.0095. Std e: 0.0743. 
4 - Kernels: 13/403 (0.03225806451612903). SV: 408/426 (0.9577464788732394). Mean e: 0.0139. Median e: 0.0066. Std e: 0.0793. 
5 - Kernels: 13/403 (0.03225806451612903). SV: 379/427 (0.8875878220140515). Mean e: 0.0147. Median e: 0.0075. Std e: 0.0790. 
6 - Kernels: 13/403 (0.03225806451612903). SV: 390/427 (0.9133489461358314). Mean e: 0.0151. Median e: 0.0083. Std e: 0.0743. 
7 - Kernels: 13/403 (0.03225806451612903). SV: 347/427 (0.8126463700234192). Mean e: 0.0138. Median e: 0.0066. S

/home/mtambos/anaconda/envs/pml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


({'fit_time': array([22.9045, 22.8748, 23.201 , 30.1612, 22.6314, 23.25  , 22.9797, 22.7122, 23.1859, 23.9337,
         22.9778, 23.0889, 22.7188, 22.8844, 23.143 , 22.8097, 22.285 , 23.5469, 22.7584, 22.7748]),
  'score_time': array([4.2227, 4.2024, 4.2244, 4.3227, 4.2968, 4.3587, 4.3644, 4.3143, 4.2249, 4.5245, 4.1892,
         4.2392, 4.2924, 4.3045, 4.3487, 4.2104, 4.2622, 4.1989, 4.3   , 4.1865]),
  'test_score': array([0.9301, 0.9225, 0.9085, 0.9366, 0.9021, 0.9014, 0.9789, 0.9014, 0.8741, 0.9507, 0.9577,
         0.9014, 0.9441, 0.9225, 0.9155, 0.8944, 0.9371, 0.9014, 0.9296, 0.9366]),
  'train_score': array([0.9202, 0.9227, 0.9251, 0.918 , 0.9249, 0.9274, 0.904 , 0.9344, 0.9272, 0.9133, 0.9087,
         0.9321, 0.9155, 0.918 , 0.9251, 0.9321, 0.9155, 0.9274, 0.9251, 0.9133])},
 [{'elapsed_time': 20.37468719482422,
   'nr_kernels_used': 13,
   'nr_sv_used': 374,
   'total_kernels': 403,
   'total_sv': 426},
  {'elapsed_time': 20.502962350845337,
   'nr_kernels_used': 13,
   'nr_

In [15]:
base_times = np.array([s['elapsed_time'] for s in base_stats])
base_kernels = np.array([s['nr_kernels_used'] for s in base_stats])
base_sv = np.array([s['nr_sv_used'] for s in base_stats])
print(
    f"Score: {base_cv_results['test_score'].mean()} +- {base_cv_results['test_score'].std()}\n"
    f"Time: {base_times.mean()} +- {base_times.std()}\n"
    f"Kernels: {base_kernels.mean()} +- {base_kernels.std()}\n"
    f"SVs: {base_sv.mean()} +- {base_sv.std()}"
)

Score: 0.9223283758495026 +- 0.02446601814854977
Time: 20.890078949928284 +- 1.6036013263561602
Kernels: 32.5 +- 84.99852939904314
SVs: 353.2 +- 42.03522332520669


# Kernel-sparse model

In [9]:
max_iter = 200
ksparse_model = BEMKL(kernels=kernels, hyp_lambda_alpha=1, hyp_lambda_beta=1,
                      hyp_gamma_alpha=1, hyp_gamma_beta=1,
                      hyp_omega_alpha=1e-11, hyp_omega_beta=1e9,
                      e_null_thrsh=1e-2, a_null_thrsh=1e-2,
                      filter_kernels=False, filter_sv=False, verbose=False,
                      max_iter=max_iter, hyperopt_enabled=False, calculate_bounds=False)
ksparse_pipeline = make_pipeline(Normalizer(), ksparse_model)

scoring.iteration = 0
scoring.stats = []
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=5)
ksparse_cv_results = cross_validate(ksparse_pipeline, X, y, cv=folds, scoring=scoring)
ksparse_stats = deepcopy(scoring.stats)
ksparse_cv_results, ksparse_stats

0 - Kernels: 3/403 (0.007444168734491315). SV: 389/426 (0.9131455399061033). Mean e: 0.0067. Median e: 0.0001. Std e: 0.0770. 
1 - Kernels: 3/403 (0.007444168734491315). SV: 384/427 (0.8992974238875878). Mean e: 0.0069. Median e: 0.0001. Std e: 0.0791. 
2 - Kernels: 3/403 (0.007444168734491315). SV: 354/427 (0.8290398126463701). Mean e: 0.0066. Median e: 0.0001. Std e: 0.0752. 
3 - Kernels: 3/403 (0.007444168734491315). SV: 374/427 (0.8758782201405152). Mean e: 0.0074. Median e: 0.0001. Std e: 0.0847. 
4 - Kernels: 3/403 (0.007444168734491315). SV: 346/426 (0.812206572769953). Mean e: 0.0071. Median e: 0.0001. Std e: 0.0808. 
5 - Kernels: 3/403 (0.007444168734491315). SV: 385/427 (0.9016393442622951). Mean e: 0.0066. Median e: 0.0001. Std e: 0.0750. 
6 - Kernels: 3/403 (0.007444168734491315). SV: 402/427 (0.9414519906323185). Mean e: 0.0074. Median e: 0.0001. Std e: 0.0847. 
7 - Kernels: 3/403 (0.007444168734491315). SV: 388/427 (0.9086651053864169). Mean e: 0.0066. Median e: 0.0001. S

/home/mtambos/anaconda/envs/pml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


({'fit_time': array([22.3753, 22.9347, 22.8556, 25.6279, 23.7483, 22.9251, 22.793 , 23.2492, 22.7382, 23.0057,
         23.1831, 22.8225, 22.8959, 25.0821, 23.1253, 23.1809, 22.8659, 22.9201, 22.9529, 22.7752]),
  'score_time': array([4.2915, 4.1726, 4.5307, 4.2984, 4.2973, 4.2126, 4.2516, 4.2946, 4.2824, 4.1953, 4.2992,
         4.2038, 4.234 , 4.3118, 4.1802, 4.2069, 4.2804, 4.222 , 4.2206, 4.2115]),
  'test_score': array([0.9301, 0.8944, 0.9577, 0.9085, 0.9021, 0.9014, 0.9296, 0.9366, 0.9161, 0.9085, 0.9225,
         0.9366, 0.9021, 0.9296, 0.9225, 0.9296, 0.9161, 0.9296, 0.9225, 0.9225]),
  'train_score': array([0.9202, 0.9274, 0.9087, 0.9297, 0.9249, 0.9274, 0.918 , 0.918 , 0.9249, 0.9251, 0.918 ,
         0.9133, 0.9225, 0.918 , 0.9227, 0.918 , 0.9155, 0.918 , 0.9227, 0.9251])},
 [{'elapsed_time': 19.98964834213257,
   'nr_kernels_used': 3,
   'nr_sv_used': 389,
   'total_kernels': 403,
   'total_sv': 426},
  {'elapsed_time': 20.44584560394287,
   'nr_kernels_used': 3,
   'nr_sv_

In [10]:
ksparse_times = np.array([s['elapsed_time'] for s in ksparse_stats])
ksparse_kernels = np.array([s['nr_kernels_used'] for s in ksparse_stats])
ksparse_sv = np.array([s['nr_sv_used'] for s in ksparse_stats])
print(
    f"Score: {ksparse_cv_results['test_score'].mean()} +- {ksparse_cv_results['test_score'].std()}\n"
    f"Time: {ksparse_times.mean()} +- {ksparse_times.std()}\n"
    f"Kernels: {ksparse_kernels.mean()} +- {ksparse_kernels.std()}\n"
    f"SVs: {ksparse_sv.mean()} +- {ksparse_sv.std()}"
)

Score: 0.9209273121244952 +- 0.014824311538043911
Time: 20.751610124111174 +- 0.7524048538477147
Kernels: 3.0 +- 0.0
SVs: 360.15 +- 46.109950119253


# SV-sparse model

In [11]:
max_iter = 200
ssparse_model = BEMKL(kernels=kernels,
                      hyp_lambda_alpha=1e-11, hyp_lambda_beta=1e9,
                      hyp_gamma_alpha=1, hyp_gamma_beta=1,
                      hyp_omega_alpha=1, hyp_omega_beta=1,
                      e_null_thrsh=1e-2, a_null_thrsh=1e-2,
                      filter_kernels=False, filter_sv=False, verbose=False,
                      max_iter=max_iter, hyperopt_enabled=False, calculate_bounds=False)
ssparse_pipeline = make_pipeline(Normalizer(), ssparse_model)

scoring.iteration = 0
scoring.stats = []
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=5)
ssparse_cv_results = cross_validate(ssparse_pipeline, X, y, cv=folds, scoring=scoring)
ssparse_stats = deepcopy(scoring.stats)
ssparse_cv_results, ssparse_stats

0 - Kernels: 13/403 (0.03225806451612903). SV: 380/426 (0.892018779342723). Mean e: 0.0154. Median e: 0.0073. Std e: 0.0881. 
1 - Kernels: 13/403 (0.03225806451612903). SV: 321/427 (0.7517564402810304). Mean e: 0.0157. Median e: 0.0089. Std e: 0.0745. 
2 - Kernels: 13/403 (0.03225806451612903). SV: 373/427 (0.8735362997658079). Mean e: 0.0154. Median e: 0.0082. Std e: 0.0786. 
3 - Kernels: 13/403 (0.03225806451612903). SV: 379/427 (0.8875878220140515). Mean e: 0.0128. Median e: 0.0057. Std e: 0.0773. 
4 - Kernels: 13/403 (0.03225806451612903). SV: 319/426 (0.7488262910798122). Mean e: 0.0142. Median e: 0.0071. Std e: 0.0780. 
5 - Kernels: 13/403 (0.03225806451612903). SV: 343/427 (0.8032786885245902). Mean e: 0.0151. Median e: 0.0073. Std e: 0.0859. 
6 - Kernels: 13/403 (0.03225806451612903). SV: 355/427 (0.8313817330210773). Mean e: 0.0161. Median e: 0.0086. Std e: 0.0816. 
7 - Kernels: 13/403 (0.03225806451612903). SV: 387/427 (0.9063231850117096). Mean e: 0.0136. Median e: 0.0066. S

/home/mtambos/anaconda/envs/pml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


({'fit_time': array([23.0085, 22.77  , 22.7981, 23.3034, 23.0207, 23.1319, 23.4263, 24.6487, 22.7371, 23.4058,
         23.1804, 22.9609, 22.8678, 23.178 , 23.1473, 23.199 , 22.7531, 22.963 , 22.9891, 22.8221]),
  'score_time': array([4.2822, 4.2535, 4.1944, 4.2481, 4.5068, 4.3159, 4.1761, 4.2258, 4.223 , 4.2619, 4.2195,
         4.2694, 4.3636, 4.2457, 4.2267, 4.2077, 4.2104, 4.3259, 4.2809, 4.3436]),
  'test_score': array([0.8881, 0.9507, 0.9296, 0.9225, 0.9301, 0.9225, 0.8873, 0.9437, 0.9161, 0.8944, 0.9507,
         0.9085, 0.9301, 0.9085, 0.9366, 0.9085, 0.9301, 0.9155, 0.9296, 0.9014]),
  'train_score': array([0.9319, 0.9157, 0.9157, 0.9204, 0.9178, 0.9157, 0.9321, 0.9157, 0.9249, 0.9274, 0.9157,
         0.9227, 0.9202, 0.9251, 0.9204, 0.9274, 0.9202, 0.9227, 0.9274, 0.9274])},
 [{'elapsed_time': 20.559281587600708,
   'nr_kernels_used': 13,
   'nr_sv_used': 380,
   'total_kernels': 403,
   'total_sv': 426},
  {'elapsed_time': 20.31490659713745,
   'nr_kernels_used': 13,
   'nr_

In [12]:
ssparse_times = np.array([s['elapsed_time'] for s in ssparse_stats])
ssparse_kernels = np.array([s['nr_kernels_used'] for s in ssparse_stats])
ssparse_sv = np.array([s['nr_sv_used'] for s in ssparse_stats])
print(
    f"Score: {ssparse_cv_results['test_score'].mean()} +- {ssparse_cv_results['test_score'].std()}\n"
    f"Time: {ssparse_times.mean()} +- {ssparse_times.std()}\n"
    f"Kernels: {ssparse_kernels.mean()} +- {ssparse_kernels.std()}\n"
    f"SVs: {ssparse_sv.mean()} +- {ssparse_sv.std()}"
)

Score: 0.9202132374667584 +- 0.018363463609058602
Time: 20.65695128440857 +- 0.4149199875959604
Kernels: 13.0 +- 0.0
SVs: 350.7 +- 31.405572753891946


In [17]:
import json
with open('wdbc_results.json', 'w') as fp:
    json.dump(
        {
            'ksparse': {
                'scores': list(ksparse_cv_results['test_score']),
                'times': list(ksparse_times),
                'kernels': [int(k) for k in ksparse_kernels],
                'svs': [int(s) for s in ksparse_sv],
            },
            'ssparse': {
                'scores': list(ssparse_cv_results['test_score']),
                'times': list(ssparse_times),
                'kernels': [int(k) for k in ksparse_kernels],
                'svs': [int(s) for s in ssparse_sv],
            },
            'base': {
                'scores': list(base_cv_results['test_score']),
                'times': list(base_times),
                'kernels': [int(k) for k in base_kernels],
                'svs': [int(s) for s in base_sv],
            },
            'total_kernels': len(kernels),
            'total_sv': len(X_train),
        },
        fp,
        indent=4,
        sort_keys=True
    )